## 데이터 및 라이브러리 로딩

In [1]:
import os

import json
import re
import pprint as ppr

### 본문 데이터 추출

In [18]:
base_dir = r"G:\내 드라이브\LAB_works\법률 LLM 프로젝트\data\데이터 전처리\2. 가공 텍스트"
upper_dir = r"\R078r3e"
source = r"\R078r3e_mainText_processed_corrected_tagged_ver2.txt"

In [19]:
# 데이터 경로
data_path = base_dir + upper_dir + source

# 텍스트 추출
with open(data_path, 'rb') as source:
    lines = source.readlines()
    lines = [element.decode('utf-8') for element in lines]

lines = lines[:len(lines)-1]

print(''.join(lines))    

1. Scope 
 This Regulation applies to vehicles of category L. [Annotation 1] 
 These categories do not include: 
(a) Vehicles with a Vmax of < 25 km/h; 
(b) Vehicles equipped for disabled riders.
[Annotation 1] 
2. Definitions 
 For the purposes of this Regulation: 
2.1. "Antilock Brake System (ABS) " means a system that senses wheel slip and automatically modulates the pressure producing the braking forces at the wheel (s) to limit the degree of wheel slip. 
2.2. "Approval of a vehicle " means the approval of a vehicle type with regard to braking. 
2.3. "Baseline test " means a stop or a series of stops carried out in order to confirm the performance of the brake prior to subjecting it to a further test such as the heating procedure or wet brake stop. 
2.4. "Brake " means those parts of the brake system where the forces opposing the movement of the vehicle are developed. 
2.5. "Brake system " means the combination of parts consisting of the control, transmission, and brake, but exclud

### 특수 데이터 추출

In [4]:
base_dir = r"G:\내 드라이브\LAB_works\법률 LLM 프로젝트\data\데이터 전처리\2. 가공 텍스트"
upper_dir = r"\R078r3e"
source_commonName = "R078r3e_mainText_spData"

In [9]:
source = rf"\{source_commonName}_tables.txt"
file_path = base_dir + upper_dir + source

flag_pathValidity = 0
if os.path.exists(file_path): flag_pathValidity = 1

dict_table = {}
if flag_pathValidity:
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()
                
            if line.startswith("Table"):
                table_number = line.split(":")[0]
                dict_table[f"{table_number}"] = []
                flag_record = 1
                continue
                
            if line.startswith("[End of Table]"):              
                flag_record = 0
                continue
                    
            if flag_record:
                dict_table[f"{table_number}"].append(line)

In [10]:
source = rf"\{source_commonName}_equations.txt"
file_path = base_dir + upper_dir + source

flag_pathValidity = 0
if os.path.exists(file_path): flag_pathValidity = 1

dict_equation = {}
if flag_pathValidity:
    with open(file_path, 'r', encoding='utf-8') as file:
        
            for line in file:
                line = line.strip()
                
                if line.startswith("Equation"):
                    equation_number = line.split(":")[0]
                    dict_equation[f"{equation_number}"] = []
                    flag_record = 1
                    continue
                
                if line.startswith("[End of Equation]"):              
                    flag_record = 0
                    continue
                    
                if flag_record:
                    dict_equation[f"{equation_number}"].append(line)

In [20]:
dict_annotation =  {}
idxl_record = []

numbering = 1
        
for idx, line in enumerate(lines):    
    if line.startswith("*"):
        dict_annotation[f"Annotation {numbering}"] = [line]
        idxl_record.append(idx)
        numbering += 1
        
for idx_removed in reversed(idxl_record):
    lines.pop(idx_removed)

In [22]:
dict_spData = {"Table":dict_table, "Equation":dict_equation, "Annotation":dict_annotation}
dict_spData = {k: v for k, v in dict_spData.items() if len(v) > 0}
ppr.pprint(dict_spData)

{'Annotation': {'Annotation 1': ['*1 As defined in the Consolidated Resolution '
                                 'on the Construction of Vehicles (R.E.3), '
                                 'document ECE/TRANS/WP.29/78/Rev.6, para. 2 – '
                                 'https://unece.org/transport/standards/transport/vehicle '
                                 '-regulations -wp29/resolutions \r\n'],
                'Annotation 2': ['*2 The distinguishing numbers of the '
                                 'Contracting Parties to the 1958 Agreement '
                                 'are reproduced in Annex 3 to the '
                                 'Consolidated Resolution on the Construction '
                                 'of Vehicles (R.E.3), document '
                                 'ECE/TRANS/WP.29/78/Rev. 6, Annex 3 '
                                 '-www.unece.org/trans/main/wp29/wp29wgs/wp29gen/wp29resolutions.html \r\n'],
                'Annotation 3': ['*3 At the time 

## 쳅터 구분

In [23]:
# 쳅터 구분

 ## 챕터 시작 지점 확인
pattern_chapter = r'^\s*(\d{1,}\.){1} .* '
idxL_chapterStart = [index for index, line in enumerate(lines) if re.match(pattern_chapter, line)]

idxL_chapterEnd = [idx_chapterStart -1 for idx_chapterStart in idxL_chapterStart]
idxL_chapterEnd.pop(0)
idxL_chapterEnd.append(len(lines)-1)

 ## 챕터 구분 및 개별 저장
linesL_chapter = []
for idx_chapterStart, idx_chapterEnd in zip(idxL_chapterStart, idxL_chapterEnd):
   lines_chapter = lines[idx_chapterStart:idx_chapterEnd+1]
   linesL_chapter.append(lines_chapter) 

## 전역 변수 정의

In [25]:
pattern_main_title = ("main", r'\s*\d{1,}\.\s+')
pattern_article = ("art", r'\s*(\d{1,}\.){2}\s+')
pattern_paragraph = ("para", r'\s*(\d{1,}\.){3}\s+')
pattern_sub_paragraph = ("paraS", r'\s*(\d{1,}\.){4}\s+')
pattern_item = ("item", r'\s*\([a-hj-u]\)\s+')
pattern_sub_item = ("itemS", r'\s*\([ivx]{1,}\)\s+')
pattern_description = ("dsc", r'\s*[A-Z]') # 추후에 수정이 필요할 수 있음

pattern_table = ("table", r'^\[Table \d{1,}\]')
pattern_equation = ("equation", r'^\[Equation \d{1,}\]')
pattern_annotation = ("annotation", r'^\[Annotation \d{1,}\]')

patternL = [pattern_main_title, pattern_article, pattern_paragraph, pattern_sub_paragraph, pattern_item, pattern_sub_item, pattern_description, pattern_table, pattern_equation, pattern_annotation]
patternL_stage = [pattern_main_title, pattern_article, pattern_paragraph, pattern_sub_paragraph]

In [26]:
dict_tagName = {"item":"Item", "itemS":"Sub-item", "dsc":"Description",
                "table":"table", "equation":"Equation", "annotation":"Annotation",
                "main":"Chapter main title",
                "art":"Article", "para":"Paragraph", "paraS":"Sub-paragraph"}

tags_stage = ["main", "art", "para", "paraS"]

tags_spData = ["table", "equation", "annotation"]

## 기능 함수 정의

### 세부 체계 구분

In [27]:
## 함수 정의

def lineTagger(linesL_input, flag_return_onlyTag=True):
    
    ## 전역 변수 로딩
    global patternL
    
    ## 행 별로 패턴 일치 여부 검색 및 결과 테깅
    tupleL_tagInfo = []
    for tag, pattern in patternL:
        pattern = r"^" + fr"{pattern}"
        tupleL_tagInfo.append([(index, tag) for index, line in enumerate(linesL_input) if re.match(pattern, line)])
    
    ## 리스트 평탄화 및 오름차순 정렬
    tupleL_tagInfo = sum(tupleL_tagInfo,[])
    tupleL_tagInfo.sort()
    
    ## 예외처리 (조건 중복, item의 알파뱃 i ~ sub item의 로마자 i)
    pattern_ish = r'^\s*\(h\)\s'
    pattern_isi = r'^\s*\(i\)\s'
    pattern_isii = r'^\s*\(ii\)\s'

    #/ Check point 1: item (h)가 존재하는가? (문제가 될 수 있는 item (i)의 선행 조건)
    #/ True: check point 2 확인
    idx_end = len(linesL_input)-1
    idxL_ish = [index for index, line in enumerate(linesL_input) if re.match(pattern_ish, line)]
    for idx_ish in idxL_ish:
   
        #// Check point 2: item (h)의 sub item이 존재하는가? (tagInfo만으로는 구분할 수 없는 요소)
        #// True: check point 3-1 확인
        if idx_ish+3 <= idx_end and re.match(pattern_isii, linesL_input[idx_ish+2]):
            idx_cursor = idx_ish + 3
            while (idx_cursor <= idx_end and tupleL_tagInfo[idx_cursor][1]=="itemS"):
                idx_cursor += 1
            #/// Check point 3: 마지막 itemS로 테그된 행이 item (i)일 수도 있지 않은가?
            #/// True: 테그 변환    
            if re.match(pattern_isi, linesL_input[idx_cursor-1]):
                tupleL_tagInfo[idx_cursor-1] = (idx_cursor-1, "item")
   
    #// False: check point 3-2 확인
        elif idx_ish+1 <= idx_end :
            idx_cursor = idx_ish + 1
            #/// Check point 3: item (h) 다음으로 itemS로 테그된 행이 item (i)이 존재하는가?
            #// True: 테그 변환
            if tupleL_tagInfo[idx_cursor][1]=="itemS":
                tupleL_tagInfo[idx_cursor] = (idx_cursor, "item")
    
    ## 결과 반환
    if flag_return_onlyTag:
        return([tag for idx, tag in tupleL_tagInfo])      
    else:
        return(tupleL_tagInfo)

### 인덱스 검색

In [28]:
def idxFinder(tags, idx_start, tag_target, tag_end=False):
    global tags_stage
    
    idxL_target = []
    idx_cursor = idx_start
    idx_end = len(tags) - 1
    #tags_out = tags_stage.copy().split()

    
    if tag_end:
        while (idx_cursor <= idx_end and tags[idx_cursor] not in tag_end):
            if tags[idx_cursor]==tag_target:
                idxL_target.append(idx_cursor)
            idx_cursor += 1
    
    else:
        while (idx_cursor <= idx_end):
            if tags[idx_cursor]==tag_target:
                idxL_target.append(idx_cursor)
            idx_cursor += 1
            
    return(idxL_target)

### 문장 페턴 반환

In [29]:
def patternFinder(target_tag, base_patternL):
    base_patternD = dict(base_patternL)
    target_pattern = base_patternD[target_tag]
    
    return(target_pattern)

### 입력 페턴을 바탕으로 문장 분해

In [30]:
def lineParser(line, break_pattern, sel_group, opt_strip=False):
    breaks = re.search(break_pattern, line)
    
    parsed_result = []   
    for idx in sel_group:
        if opt_strip:
            parsed_result.append(breaks.group(idx).strip())
        else:
            parsed_result.append(breaks.group(idx))
            
    return(parsed_result)

### JSON 구성 요소 생성 및 입력

In [31]:
## 독립형 sublistLoader (미사용)

def sublistLoader(lines, tags, base_dict, target_tag, pivot_idx, opt_parse=False, opt_strip=False):
    global patterL
    global dict_tagName
    global tags_stage

    tag_pivot = tags[pivot_idx]
    tags_end = tags_stage.copy()
    tags_end.append(tag_pivot)

    idxL_loader = idxFinder(tags, pivot_idx+1, target_tag, tag_end=tags_end)
    ##        
    lines_loader = []
    for idx_loader in idxL_loader:
        lines_loader.append(lines[idx_loader].strip())

    ##
    if len(lines_loader):
        if opt_parse:
            pattern_pivot = patternFinder(tag_pivot, patterL)
            ele_searchingKey = lineParser(lines[pivot_idx], rf"({pattern_pivot})" + r"(.*)", sel_group=[3], opt_strip=opt_strip)[0]
        else:
            ele_searchingKey = lines[pivot_idx].strip()
            
        idx_pivotInlist = base_dict[fullName_pivot].index(ele_searchingKey)

        fullName_pivot = dict_tagName[tag_pivot]
        base_dict[fullName_pivot][idx_pivotInlist].append(lines_loader)

In [32]:
def listLoader(lines, tags, base_dict, target_tag, pivot_idx, target_tag_subList=None):
    global dict_tagName
    global tags_stage
    
    idxL_loader = idxFinder(tags, pivot_idx+1, target_tag, tag_end=tags_stage)
    if target_tag_subList:
        idx_end = len(lines)-1
        idxL_with_subList = [idx_loader for idx_loader in idxL_loader if idx_loader < idx_end and tags[idx_loader+1]==target_tag_subList]
    ##        
    lines_loader = []
    for idx_loader in idxL_loader:
        if target_tag_subList and idx_loader in idxL_with_subList:
            lines_subLoader = []
            idxL_subLoader = idxFinder(tags, idx_loader+1, target_tag_subList, tag_end=target_tag)
            for idx_subLoader in idxL_subLoader: lines_subLoader.append(lines[idx_subLoader].strip())
            
            lines_loader.append([lines[idx_loader].strip(), lines_subLoader])
            
        else:     
            lines_loader.append(lines[idx_loader].strip())

    ##
    if len(lines_loader):
        fullName = dict_tagName[target_tag]
        base_dict[fullName] = lines_loader

In [33]:
def listLoader_spData(lines, tags, base_dict, target_tag, pivot_idx):
    global tags_stage
    global dict_spData
    
    idxL_loader = idxFinder(tags, pivot_idx+1, target_tag, tag_end=tags_stage)
    
    info_spData = []
    for idx_loader in idxL_loader:
        pattern_test = r'^\[([a-zA-Z]*) (\d{1,})\]'
        info_spData.append(lineParser(lines[idx_loader], pattern_test, [1,2], opt_strip=True))
    
    if info_spData:
        for dataType, numbering in info_spData:
            base_dict[f"{dataType} {numbering}"] = dict_spData[f"{dataType}"][f"{dataType} {numbering}"]

In [34]:
def dictLoader(lines, tags, base_dict, pivot_pattern, pivot_idx):
    global tags_spData
    
    numbering, dsc_0 = lineParser(lines[pivot_idx], rf"({pivot_pattern})" + r"(.*)", sel_group=[1,3], opt_strip=True)
    dict_loaded = base_dict[numbering] = {}
    
    listLoader(lines, tags, dict_loaded, target_tag="dsc", pivot_idx=pivot_idx)
    if "Description" in list(dict_loaded.keys()): dict_loaded["Description"].insert(0,dsc_0)
    else: dict_loaded["Description"] = [dsc_0]
    
    listLoader(lines, tags, dict_loaded, target_tag="item", target_tag_subList="itemS", pivot_idx=pivot_idx)
            
    for dataType in tags_spData:
        listLoader_spData(lines, tags, dict_loaded, target_tag=dataType, pivot_idx=pivot_idx)
        
    return(dict_loaded)

In [35]:
def recursive_dictLoader(lines, tags, base_dict, init_idx, parental_tag):
    global patternL_stage
    global patternL
    
    tags_stage = [tag for tag, pattern in patternL_stage]
    
    idxL_parentalTag = idxFinder(tags=tags, idx_start=init_idx, tag_target=parental_tag, tag_end=tags_stage[:tags_stage.index(parental_tag)])
    pattern_parentalTag = patternFinder(parental_tag, patternL)
    
    for idx_parentalTag in idxL_parentalTag:
        
        dict_parental = dictLoader(lines, tags, base_dict, pattern_parentalTag, idx_parentalTag)
        
        if parental_tag != tags_stage[-1]:
            idx_parentalStage = tags_stage.index(parental_tag)
            tag_child = tags_stage[idx_parentalStage+1]
            recursive_dictLoader(lines, tags, dict_parental, idx_parentalTag+1, tag_child)

In [36]:
def dictInitiator_chapter(lines, tags, main_line_idx):
    global tags_spData
    
    dict_return = {}
    line_main = lines_chapter[main_line_idx]
    num_chapter, title_chapter = [elm.strip() for elm in line_main.split('.')]
    dict_return["Chapter"] = num_chapter
    dict_return["Title"] = title_chapter
    
    listLoader(lines, tags, dict_return, target_tag="dsc", pivot_idx=main_line_idx)
    listLoader(lines, tags, dict_return, target_tag="item", target_tag_subList="itemS", pivot_idx=main_line_idx)
    
    for dataType in tags_spData:
        listLoader_spData(lines, tags, dict_return, target_tag=dataType, pivot_idx=main_line_idx)
    
    return dict_return

## 워킹 스페이스

### 개별 쳅터

In [37]:
## 재귀 함수 이용

#
lines_chapter = linesL_chapter[1]
tags_chapter = lineTagger(lines_chapter, flag_return_onlyTag=True)

#     
dict_chapter = dictInitiator_chapter(lines_chapter, tags_chapter, main_line_idx=0)

#
recursive_dictLoader(lines_chapter, tags_chapter, dict_chapter, init_idx=1, parental_tag="art")

In [24]:
## 명시적 반복문 이용 (참고용)

lines_chapter = linesL_chapter[4]
tags_chapter = lineTagger(lines_chapter, flag_return_onlyTag=True)

#     
dict_chapter = {}

#
line_main = lines_chapter[0]
num_chapter, title_chapter = [elm.strip() for elm in line_main.split('.')]

dict_chapter["Chapter"] = num_chapter
dict_chapter["Title"] = title_chapter

#
listLoader(lines_chapter, tags_chapter, dict_chapter, target_tag="dsc", pivot_idx=0)
listLoader(lines_chapter, tags_chapter, dict_chapter, target_tag="item", target_tag_subList="itemS", pivot_idx=0)

#
idxL_art = idxFinder(tags=tags_chapter, idx_start=0, tag_target="art")
pattern_art = patternFinder("art", patternL)
for idx_art in idxL_art:
    
    dict_art = dictLoader(lines_chapter, tags_chapter, dict_chapter, pattern_art, idx_art, opt_dsc=True, opt_item=True, opt_itemS=True)
    
    idxL_para = idxFinder(tags=tags_chapter, idx_start=idx_art+1, tag_target="para", tag_end="art")
    pattern_para = patternFinder("para", patternL)
    for idx_para in idxL_para:    

        dict_para = dictLoader(lines_chapter, tags_chapter, dict_art, pattern_para, idx_para, opt_dsc=True, opt_item=True, opt_itemS=True)
        
        idxL_paraS = idxFinder(tags=tags_chapter, idx_start=idx_para+1, tag_target="paraS", tag_end=["art", "para"])
        pattern_paraS = patternFinder("paraS", patternL)
        for idx_paraS in idxL_paraS:
            
            dictLoader(lines_chapter, tags_chapter, dict_para, pattern_paraS, idx_paraS, opt_dsc=True, opt_item=True, opt_itemS=True)

In [112]:
ppr.pprint(dict_chapter)

{'2.1.': {'Description': ['Engine idle speed',
                          "The engine idle speed is set to the manufacturer's "
                          'specification.']},
 '2.2.': {'Description': ['Tyre pressures',
                          "The tires are inflated to the manufacturer's "
                          'specification for the vehicle loading condition for '
                          'the test.']},
 '2.3.': {'Description': ['Control application points and direction',
                          'For a hand control lever, the input force (F) is '
                          "applied on the control lever's forward surface "
                          'perpendicular to the axis of the lever fulcrum and '
                          'its outermost point on the plane along which the '
                          'control lever rotates (see Figure 1 below). The '
                          'input force is applied to a point located 50 mm '
                          'from the outermost point

### 문서 전체

In [41]:
## 재귀 함수 이용
dict_full = {}
dict_full["Resource"] = "Addendum 77: UN Regulation No. 78 Main Text"
dict_full["Title"] = "Agreement: Concerning the Adoption of Harmonized Technical United Nations Regulations for Wheeled Vehicles, Equipment and Parts which can be Fitted and/or be Used on Wheeled Vehicles and the Conditions for Reciprocal Recognition of Approvals Granted on the Basis of these United Nations Regulations"
dict_full["Version"] = "Revision 3"

for lines_chapter in linesL_chapter:
    tags_chapter = lineTagger(lines_chapter, flag_return_onlyTag=True)

    #       
    dict_chapter = dictInitiator_chapter(lines_chapter, tags_chapter, main_line_idx=0)
    num_chapter = dict_chapter['Chapter']

    #
    recursive_dictLoader(lines_chapter, tags_chapter, dict_chapter, init_idx=1, parental_tag="art")
    
    #
    dict_full[f"chapter {num_chapter}"] = dict_chapter

In [42]:
ppr.pprint(dict_full)

{'Resource': 'Addendum 77: UN Regulation No. 78 Main Text',
 'Title': 'Agreement: Concerning the Adoption of Harmonized Technical United '
          'Nations Regulations for Wheeled Vehicles, Equipment and Parts which '
          'can be Fitted and/or be Used on Wheeled Vehicles and the Conditions '
          'for Reciprocal Recognition of Approvals Granted on the Basis of '
          'these United Nations Regulations',
 'Version': 'Revision 3',
 'chapter 1': {'Annotation 1': ['*1 As defined in the Consolidated Resolution '
                                'on the Construction of Vehicles (R.E.3), '
                                'document ECE/TRANS/WP.29/78/Rev.6, para. 2 – '
                                'https://unece.org/transport/standards/transport/vehicle '
                                '-regulations -wp29/resolutions \r\n'],
               'Chapter': '1',
               'Description': ['This Regulation applies to vehicles of '
                               'category L. [An

## JSON 직렬화 처리 후 파일 저장

### 데이터 경로

In [43]:
base_dir = r"G:\내 드라이브\LAB_works\법률 LLM 프로젝트\data\데이터 전처리\3. JSON 컨버팅\JSON_byChapter_v2"
upper_dir = r"\R078r3e_mainText"
fileName_common = r"\R078r3e_mainText"

In [44]:
folder_path = base_dir + upper_dir

### 문서 전체

In [45]:
with open(folder_path+fileName_common+"_fullchpter_ver2.json", "w") as file:
    json.dump(dict_full, file)

### 개별 챕터

In [46]:
keys_chapter = list(filter(lambda x: "chapter" in x, list(dict_full.keys())))

for number_of_chapter in range(len(linesL_chapter)):
    with open(folder_path+fileName_common+f"_chapter{number_of_chapter+1}_ver2.json", "w") as file:
        json.dump(dict_full[keys_chapter[number_of_chapter]], file)